# Test ETFA

In [1]:
from aiida import load_profile

load_profile()


Profile<uuid='26d1506a5df343b38cc9d3e4cc459793' name='default'>

In [2]:
from workgraph_collections.cp2k.xps import xps_workflow
wg = xps_workflow()
wg

ndata:  {'node_type': 'CALCFUNCTION', 'executor': <function get_structures at 0x7fe98efb3130>}
ndata:  {'node_type': 'CALCFUNCTION', 'executor': <function binding_energy at 0x7fe98efb3a30>}


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'binding…

## Prepare the input files

In [3]:
from ase.build import fcc111
from ase.io import  read
from ase.visualize import view
from aiida.orm import Dict, load_code, load_node, StructureData

cp2k_code = load_code("cp2k-ssmp-2024.1@localhost")
# cp2k_code = load_code("cp2k-psmp-2024.1@eiger")
basis_file = load_node(9041)
pseudo_file = load_node(9042)
basis_pseudo_files = {
            "basis": basis_file,
            "pseudo": pseudo_file,
        }
# Structure.
etfa = read("datas/ETFA.xyz")
# view(pt_111_333)
structure = StructureData(ase=etfa)
structure.label = "ETFA molecule, vacuum 5.0 Angstroms"
# Parameters.
parameters = {
        "GLOBAL": {
            "RUN_TYPE": "ENERGY_FORCE",
        },
        "FORCE_EVAL": {
            "METHOD": "Quickstep",
            "DFT": {
                "BASIS_SET_FILE_NAME": "BASIS_MOLOPT",
                "POTENTIAL_FILE_NAME": "POTENTIALS",
                "UKS": True,
                "MULTIPLICITY": 1,
                "SCF": {
                    "EPS_DIIS": 0.1,
                    "EPS_SCF": 1e-06,
                    "ADDED_MOS": 200,
                    "OUTER_SCF": {
                        "MAX_SCF": 20,
                        "EPS_SCF": 1e-06,
                    },
                    "OT": {
                        "ORTHO_IRAC": "CHOL",
                        "N_HISTORY_VEC": 7,
                        "SAFE_DIIS": False,
                        "PRECONDITIONER": "FULL_ALL",
                        "ENERGY_GAP": 0.05,
                        "MINIMIZER": "CG",
                        "ALGORITHM": "IRAC",
                        "EPS_IRAC_SWITCH": 0.01,
                    },
                },
                "QS": {
                    "METHOD": "GPW",
                    "EPS_DEFAULT": 1.0e-12,
                    "EXTRAPOLATION_ORDER": 3,
                },
                "MGRID": {
                    "NGRIDS": 5,
                    "CUTOFF": 1000,
                    "REL_CUTOFF": 50,
                },
                "XC": {
                    "XC_FUNCTIONAL": {
                        "PBE": {
                        "PARAMETRIZATION": "PBESOL",
                        }
                    },
                },
            },
            "SUBSYS": {
                "KIND": [
                    {
                        "_": "O",
                        "BASIS_SET": "DZVP-MOLOPT-GTH",
                        "POTENTIAL": "GTH-PBESOL-q6",
                    },
                    {
                        "_": "C",
                        "BASIS_SET": "DZVP-MOLOPT-GTH",
                        "POTENTIAL": "GTH-PBESOL-q4",
                    },
                    {
                        "_": "H",
                        "BASIS_SET": "DZVP-MOLOPT-GTH",
                        "POTENTIAL": "GTH-PBE",
                    },
                    {
                        "_": "Pt",
                        "ELEMENT": "Pt",
                        "BASIS_SET": "DZVP-MOLOPT-SR-GTH-q18",
                        "POTENTIAL": "GTH-PBESOL-q18",
                    },
                ],
            },
        },
    }
metadata = {"options": {'custom_scheduler_commands' : 'export OMP_NUM_THREADS=4',
                    'resources': {
                                'num_machines' : 1,
                                'num_mpiprocs_per_machine' : 1,
                                }
                    }
        }
metadata_eiger = {"options": {
                  'custom_scheduler_commands' : '#SBATCH --account=mr32',
                  'resources': {
                              'num_machines' : 1,
                              'num_mpiprocs_per_machine' : 128,
                              }
                  }
                  }
#---------------------------------------------------------------
core_hole_pseudos = {
    "C": {
            "_": "X",
            "ELEMENT": "C",
            "BASIS_SET": "DZVP-MOLOPT-GTH",
            "POTENTIAL": "GTH-PBESOL-q5_1s1",
            "CORE_CORRECTION": 1,
        },
    "O": {
            "_": "X",
            "ELEMENT": "O",
            "BASIS_SET": "DZVP-MOLOPT-GTH",
            "POTENTIAL": "GTH-PBESOL-q7_1s1",
            "CORE_CORRECTION": 1,
        },
    "Pt": {
            "_": "X",
            "ELEMENT": "Pt",
            "BASIS_SET": "DZVP-MOLOPT-SR-GTH-q18",
            "POTENTIAL": "GTH-PBESOL-q19_4f13",
            "CORE_CORRECTION": 1,
        }
}
wg = xps_workflow()
wg.name = "ETFA"
wg.nodes["get_structures"].set({
        "structure": structure,
        "atoms_list": [(0, "1s"), (1, "1s"), (2, "1s"), (3, "1s")],
        })
wg.nodes["run_scf"].set({
      "parameters": parameters,
      "code": cp2k_code,
      "basis_pseudo_files": basis_pseudo_files,
      "core_hole_pseudos": core_hole_pseudos,
    #   "metadata": metadata_eiger,
      "metadata": metadata,
    })
wg.nodes["binding_energy"].set({
    "corrections": {
        "C": 397.4,
        "O": 746.4,
        "Pt": 460.6,
        "Au": 296.1,
        }})
# wg.submit()
wg.run()
print("Binding energy of Pt (111) surface: ", wg.nodes["binding_energy"].outputs["result"].value)



FileNotFoundError: [Errno 2] No such file or directory: 'datas/ETFA.xyz'

## pt

In [3]:
from ase.build import fcc111
from ase.visualize import view
from aiida.orm import Dict, load_code, load_node, StructureData

# cp2k_code = load_code("cp2k-ssmp-2024.1@localhost")
cp2k_code = load_code("cp2k-psmp-2024.1@eiger")
basis_file = load_node(7897)
pseudo_file = load_node(7898)
basis_pseudo_files = {
            "basis": basis_file,
            "pseudo": pseudo_file,
        }
# Structure.
pt_111_333 = fcc111("Pt", size=(3, 3, 3), vacuum=5.0)
pt_111_333.set_tags(None)
# view(pt_111_333)
structure_111_333 = StructureData(ase=pt_111_333)
structure_111_333.label = "Pt (111), 3x3x3, vacuum 5.0 Angstroms"
# Parameters.
parameters = {
        "GLOBAL": {
            "RUN_TYPE": "ENERGY_FORCE",
        },
        "FORCE_EVAL": {
            "METHOD": "Quickstep",
            "DFT": {
                "BASIS_SET_FILE_NAME": "BASIS_MOLOPT",
                "POTENTIAL_FILE_NAME": "POTENTIALS",
                "UKS": True,
                "MULTIPLICITY": 1,
                "SCF": {
                    "EPS_DIIS": 0.1,
                    "EPS_SCF": 1e-06,
                    "ADDED_MOS": 200,
                    "OUTER_SCF": {
                        "MAX_SCF": 20,
                        "EPS_SCF": 1e-06,
                    },
                    "DIAGONALIZATION": {
                        "ALGORITHM": "STANDARD",
                    },
                    "SMEAR": {
                        "METHOD": "FERMI_DIRAC",
                        "ELECTRONIC_TEMPERATURE": 500,
                    },
                    "MIXING": {
                        "METHOD": "BROYDEN_MIXING",
                        "ALPHA": 0.1,
                        "BETA": 1.5,
                        "NBUFFER": 8,
                    }
                },
                "QS": {
                    "METHOD": "GPW",
                    "EPS_DEFAULT": 1.0e-12,
                    "EXTRAPOLATION_ORDER": 3,
                },
                "MGRID": {
                    "NGRIDS": 5,
                    "CUTOFF": 1000,
                    "REL_CUTOFF": 50,
                },
                "XC": {
                    "XC_FUNCTIONAL": {
                        "PBE": {
                        "PARAMETRIZATION": "PBESOL",
                        }
                    },
                },
            },
            "SUBSYS": {
                "KIND": [
                    {
                        "_": "Pt",
                        "ELEMENT": "Pt",
                        "BASIS_SET": "DZVP-MOLOPT-SR-GTH-q18",
                        "POTENTIAL": "GTH-PBESOL-q18",
                    },
                ],
            },
        },
    }
metadata = {"options": {'custom_scheduler_commands' : 'export OMP_NUM_THREADS=4',
                    'resources': {
                                'num_machines' : 1,
                                'num_mpiprocs_per_machine' : 1,
                                }
                    }
        }
metadata_eiger = {"options": {
                  'custom_scheduler_commands' : '#SBATCH --account=mr32',
                  'resources': {
                              'num_machines' : 1,
                              'num_mpiprocs_per_machine' : 128,
                              }
                  }
                  }
#---------------------------------------------------------------
core_hole_pseudos = {
    "Pt": {
            "_": "X",
            "ELEMENT": "Pt",
            "BASIS_SET": "DZVP-MOLOPT-SR-GTH-q18",
            "POTENTIAL": "GTH-PBESOL-q19_4f13",
            "CORE_CORRECTION": 1,
        }
}
wg = xps_workflow()
wg.name = "binging_energy_Pt_111"
wg.nodes["get_structures"].set({
        "structure": structure_111_333,
        "atoms_list": [(0, "4f"), (1, "4f")],
        })
wg.nodes["run_scf"].set({
      "parameters": parameters,
      "code": cp2k_code,
      "basis_pseudo_files": basis_pseudo_files,
      "core_hole_pseudos": core_hole_pseudos,
      "metadata": metadata_eiger,
    })
wg.nodes["binding_energy"].set({
    "corrections": {
        "C": 397.4,
        "O": 746.4,
        "Pt": 460.6,
        "Au": 296.1,
        }})
# wg.submit()
wg.run()
print("Binding energy of Pt (111) surface: ", wg.nodes["binding_energy"].outputs["result"].value)



ndata:  {'node_type': 'CALCFUNCTION', 'executor': <function binding_energy at 0x7f3058f9fac0>}


05/13/2024 01:31:34 PM <1602650> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9019|WorkGraph|continue_workgraph]: Continue workgraph.


Continue workgraph.


05/13/2024 01:31:34 PM <1602650> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9019|WorkGraph|continue_workgraph]: nodes ready to run: get_structures
05/13/2024 01:31:34 PM <1602650> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9019|WorkGraph|run_nodes]: Run node: get_structures, type: CALCFUNCTION


------------------------------------------------------------
executor:  <function get_structures at 0x7f304732d990>
node type: calcfunction/workfunction.


...

Continue workgraph.


05/13/2024 01:57:09 PM <1602650> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [9019|WorkGraph|finalize]: Finalize


node:  binding_energy FAILED
node:  get_structures FINISHED
node:  run_scf FAILED
is workgraph finished:  True
group outputs:  []
    Node binding_energy failed.
Binding energy of Pt (111) surface:  None


05/13/2024 01:57:20 PM <1602650> aiida.parser.Cp2kBaseParser: [WARNING] No restart file found in the retrieved folder.
05/13/2024 01:57:20 PM <1602650> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [WARNING] output parser returned exit code<304>: The output file contains the word "ABORT".


on awaitable finished:  scf_Pt_1
update node state:  scf_Pt_1


In [4]:
print("Binding energy of Pt bulk: ", wg.nodes["binding_energy"].outputs["result"].value)


Binding energy of Pt bulk:  None
